In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import glob
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
import segmentation_models as sm
sm.set_framework('tf.keras')
import multiprocessing as mp
from save_weights_every_epoch import CallbackForSavingModelWeights

Segmentation Models: using `keras` framework.


In [2]:
csv_file = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [3]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [4]:
# def check_valid_paths(path):
#     try:
#         img = tf.io.read_file(path)
#         img = tf.image.decode_jpeg(img, channels=3)
#         img = tf.image.resize(img, size=(256, 256))
#         return 'DONE'
#     except:
#         return path

In [5]:
# paths = csv_file['fixed_inputs'].values.tolist()

In [6]:
# issues = []
# for x in tqdm(paths):
#     issues.append(check_valid_paths(x))

In [7]:
for x in csv_file['fixed_inputs'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

In [8]:
for x in csv_file['mask_paths'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

In [9]:
# for x in issues:
#     if x != 'DONE':
#         print(x)

In [10]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../ml-data-training/ship_segmentation_data/train_v2/6384c3e78.jpg']

Input and masks path checks are done.

In [11]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [12]:
allowed_gpus = [0, 1, 2, 3, 4, 5]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5')


In [13]:
print(REPLICAS)

6


In [14]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [15]:
load_dotenv('../config_files/dev.env')

True

In [16]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size)
    train, val = train_test_split(train, test_size = test_size)
    return train, val, test

In [17]:
train, val, test = split_datasets(csv_file)

In [18]:
def read_train_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels=1)
    img = tf.image.resize(img, size=shape)
    mask = tf.image.resize(mask, size=shape)
    img = img / 255
    mask = mask / 255
    return img, mask

def get_data(data, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_train_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [19]:
train_dataset = get_data(train, shape = (32, 32), batch_size=32)
val_dataset = get_data(val, shape = (32, 32), batch_size=32)

In [20]:
def get_model(model_name):
    with strategy.scope():
        model = sm.Unet(model_name)
        model.compile(
            'SGD',
            loss = sm.losses.bce_jaccard_loss,
            metrics = [sm.metrics.iou_score]
        )
    return model

In [21]:
os.environ['tb_path']

'/home/ubuntu/ship_segmentation/TB/'

In [36]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/res50_baseline_bs_128_is_32/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (32, 32), batch_size=128)
val_dataset = get_data(val, shape = (32, 32), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
1474/1474 [==============================] - 177s 60ms/step - loss: 1.0267 - iou_score: 0.0011 - val_loss: 1.0095 - val_iou_score: 0.0012
Epoch 2/50
1474/1474 [==============================] - 86s 58ms/step - loss: 1.0076 - iou_score: 0.0020 - val_loss: 1.0000 - val_iou_score: 0.0088
Epoch 3/50
1474/1474 [==============================] - 86s 58ms/step - loss: 0.8958 - iou_score: 0.1127 - val_loss: 0.7729 - val_iou_score: 0.2353
Epoch 4/50
1474/1474 [==============================] - 86s 58ms/step - loss: 0.7270 - iou_score: 0.2811 - val_loss: 0.7060 - val_iou_score: 0.3024
Epoch 5/50
1474/1474 [==============================] - 85s 58ms/step - loss: 0.6750 - iou_score: 0.3331 - val_loss: 0.6667 - val_iou_score: 0.3415
Epoch 6/50
1474/1474 [==============================] - 84s 57ms/step - loss: 0.6447 - iou_score: 0.3632 - val_loss: 0.6695 - val_iou_score: 0.3389
Epoch 7/50
1474/1474 [==============================] - 84s 57ms/step - loss: 0.6165 - iou_score: 0.3912 - val_

In [39]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_64/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/res50_baseline_bs_128_is_64/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (64, 64), batch_size=128)
val_dataset = get_data(val, shape = (64, 64), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
1474/1474 [==============================] - 100s 65ms/step - loss: 1.0268 - iou_score: 0.0015 - val_loss: 1.0078 - val_iou_score: 0.0025
Epoch 2/50
1474/1474 [==============================] - 96s 65ms/step - loss: 0.9002 - iou_score: 0.1061 - val_loss: 0.7927 - val_iou_score: 0.2130
Epoch 3/50
1474/1474 [==============================] - 96s 65ms/step - loss: 0.6883 - iou_score: 0.3165 - val_loss: 0.5719 - val_iou_score: 0.4355
Epoch 4/50
1474/1474 [==============================] - 96s 65ms/step - loss: 0.5315 - iou_score: 0.4746 - val_loss: 0.5262 - val_iou_score: 0.4811
Epoch 5/50
1474/1474 [==============================] - 96s 65ms/step - loss: 0.4969 - iou_score: 0.5092 - val_loss: 0.4947 - val_iou_score: 0.5123
Epoch 6/50
1474/1474 [==============================] - 96s 65ms/step - loss: 0.4769 - iou_score: 0.5291 - val_loss: 0.4723 - val_iou_score: 0.5343
Epoch 7/50
1474/1474 [==============================] - 96s 65ms/step - loss: 0.4524 - iou_score: 0.5533 - val_

In [40]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_128/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/res50_baseline_bs_128_is_128/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (128, 128), batch_size=128)
val_dataset = get_data(val, shape = (128, 128), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
1474/1474 [==============================] - 286s 191ms/step - loss: 0.9373 - iou_score: 0.0842 - val_loss: 0.7433 - val_iou_score: 0.2610
Epoch 2/50
1474/1474 [==============================] - 281s 191ms/step - loss: 0.6078 - iou_score: 0.3955 - val_loss: 0.5180 - val_iou_score: 0.4852
Epoch 3/50
1474/1474 [==============================] - 282s 191ms/step - loss: 0.4130 - iou_score: 0.5910 - val_loss: 0.3980 - val_iou_score: 0.6064
Epoch 4/50
1474/1474 [==============================] - 281s 191ms/step - loss: 0.3652 - iou_score: 0.6389 - val_loss: 0.3712 - val_iou_score: 0.6339
Epoch 5/50
1474/1474 [==============================] - 281s 191ms/step - loss: 0.3406 - iou_score: 0.6635 - val_loss: 0.3367 - val_iou_score: 0.6677
Epoch 6/50
1474/1474 [==============================] - 282s 191ms/step - loss: 0.3275 - iou_score: 0.6764 - val_loss: 0.3384 - val_iou_score: 0.6663
Epoch 7/50
1474/1474 [==============================] - 282s 191ms/step - loss: 0.3127 - iou_score: 

In [41]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/incep_baseline_bs_128_is_32/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_128_is_32/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (32, 32), batch_size=128)
val_dataset = get_data(val, shape = (32, 32), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('inceptionv3')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

87910968/87910968 [==============================] - 1s 0us/step
Epoch 1/50
1474/1474 [==============================] - 96s 60ms/step - loss: 1.0262 - iou_score: 0.0019 - val_loss: 1.0050 - val_iou_score: 0.0045
Epoch 2/50
1474/1474 [==============================] - 87s 59ms/step - loss: 0.9442 - iou_score: 0.0631 - val_loss: 0.8099 - val_iou_score: 0.1968
Epoch 3/50
1474/1474 [==============================] - 86s 59ms/step - loss: 0.7182 - iou_score: 0.2890 - val_loss: 0.6848 - val_iou_score: 0.3227
Epoch 4/50
1474/1474 [==============================] - 86s 58ms/step - loss: 0.6449 - iou_score: 0.3625 - val_loss: 0.6651 - val_iou_score: 0.3424
Epoch 5/50
1474/1474 [==============================] - 85s 58ms/step - loss: 0.6063 - iou_score: 0.4010 - val_loss: 0.6295 - val_iou_score: 0.3782
Epoch 6/50
1474/1474 [==============================] - 85s 58ms/step - loss: 0.5815 - iou_score: 0.4257 - val_loss: 0.6269 - val_iou_score: 0.3810
Epoch 7/50
1474/1474 [=========================

In [42]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/incep_baseline_bs_128_is_64/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_128_is_64/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (64, 64), batch_size=128)
val_dataset = get_data(val, shape = (64, 64), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('inceptionv3')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
1474/1474 [==============================] - 103s 65ms/step - loss: 1.0238 - iou_score: 0.0047 - val_loss: 0.9715 - val_iou_score: 0.0369
Epoch 2/50
1474/1474 [==============================] - 93s 63ms/step - loss: 0.6488 - iou_score: 0.3569 - val_loss: 0.5732 - val_iou_score: 0.4333
Epoch 3/50
1474/1474 [==============================] - 94s 64ms/step - loss: 0.5157 - iou_score: 0.4900 - val_loss: 0.5157 - val_iou_score: 0.4905
Epoch 4/50
1474/1474 [==============================] - 94s 63ms/step - loss: 0.4726 - iou_score: 0.5330 - val_loss: 0.4760 - val_iou_score: 0.5301
Epoch 5/50
1474/1474 [==============================] - 95s 65ms/step - loss: 0.4472 - iou_score: 0.5582 - val_loss: 0.4573 - val_iou_score: 0.5488
Epoch 6/50
1474/1474 [==============================] - 93s 63ms/step - loss: 0.4290 - iou_score: 0.5763 - val_loss: 0.4476 - val_iou_score: 0.5582
Epoch 7/50
1474/1474 [==============================] - 95s 64ms/step - loss: 0.4149 - iou_score: 0.5903 - val_

In [43]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/incep_baseline_bs_128_is_128/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_128_is_128/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (128, 128), batch_size=128)
val_dataset = get_data(val, shape = (128, 128), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('inceptionv3')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
1474/1474 [==============================] - 242s 159ms/step - loss: 1.0294 - iou_score: 0.0018 - val_loss: 1.0004 - val_iou_score: 0.0099
Epoch 2/50
1474/1474 [==============================] - 234s 158ms/step - loss: 0.7739 - iou_score: 0.2301 - val_loss: 0.6678 - val_iou_score: 0.3353
Epoch 3/50
1474/1474 [==============================] - 234s 159ms/step - loss: 0.5364 - iou_score: 0.4667 - val_loss: 0.4791 - val_iou_score: 0.5244
Epoch 4/50
1474/1474 [==============================] - 234s 159ms/step - loss: 0.4348 - iou_score: 0.5683 - val_loss: 0.4243 - val_iou_score: 0.5796
Epoch 5/50
1474/1474 [==============================] - 235s 159ms/step - loss: 0.3873 - iou_score: 0.6158 - val_loss: 0.3946 - val_iou_score: 0.6090
Epoch 6/50
1474/1474 [==============================] - 234s 159ms/step - loss: 0.3593 - iou_score: 0.6438 - val_loss: 0.3797 - val_iou_score: 0.6237
Epoch 7/50
1474/1474 [==============================] - 234s 159ms/step - loss: 0.3378 - iou_score: 

In [24]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/incep_baseline_bs_64_is_256/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_64_is_256/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (256, 256), batch_size=64)
val_dataset = get_data(val, shape = (256, 256), batch_size=64, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('inceptionv3')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (64 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50


In [ ]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_64_is_256/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/res50_baseline_bs_64_is_256/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (256, 256), batch_size=64)
val_dataset = get_data(val, shape = (256, 256), batch_size=64, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (64 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

In [ ]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/incep_baseline_bs_64_is_512/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_64_is_512/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (512, 512), batch_size=64)
val_dataset = get_data(val, shape = (512, 512), batch_size=64, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('inceptionv3')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (64 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

In [ ]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_64_is_512/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/segmentation/res50_baseline_bs_64_is_512/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (512, 512), batch_size=64)
val_dataset = get_data(val, shape = (512, 512), batch_size=64, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (64 * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )